## Building flask api

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')

In [3]:
from src.serve import *

In [4]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [5]:
data_dir = 'data/midi/v12/'

file_path = Path('../../')
path = file_path/data_dir/'midi_encode'
out_path = file_path/'data/generated/'

In [6]:
encoded_path = file_path/'data/assets/v12/seed'

## Learn config

In [7]:
import hashlib
import shutil

In [8]:
def df2records(path):
    df = pd.read_csv(path/'midi_encode.csv')
    df = df.loc[df['numpy'].notna()] # make sure it exists
    df = df.loc[df.source == 'hooktheory_c'] # hooktheory only
    df = df.reindex(index=df.index[::-1]) # A's first
    df = df.where((pd.notnull(df)), None) # nan values break json
    return df.to_dict('records')

In [9]:
df2records(path)

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (0,2,6,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


[{'section': 'intro',
  'ht_mode': 1.0,
  'mxl': None,
  'artist': 'adam-neely',
  'midi': 'midi_sources/hooktheory/pianoroll/a/adam-neely/adam-neelys-bass-lessons/intro_key_cmajor.mid',
  'genres': 'Experimental',
  'midi_title': "Adam Neely's Bass Lessons",
  'ht_offset': 0.0,
  'parts': 'intro',
  'numpy': 'midi_encode/hooktheory/pianoroll/a/adam-neely/adam-neelys-bass-lessons/intro_key_cmajor.npy',
  'song_url': 'https://www.hooktheory.com/theorytab/view/adam-neely/adam-neelys-bass-lessons',
  'ht_key': 'C',
  'ht_time_signature': 4.0,
  'ht_bpm': 123.0,
  'title': 'adam-neelys-bass-lessons',
  'source': 'hooktheory_c',
  'md5': '8bcab6714f50bf509b7303478988636e'},
 {'section': 'chorus',
  'ht_mode': 1.0,
  'mxl': None,
  'artist': 'asher-roth',
  'midi': 'midi_sources/hooktheory/pianoroll/a/asher-roth/i-love-college/chorus_key_cmajor.mid',
  'genres': 'Hip-Hop/Rap',
  'midi_title': 'I Love College',
  'ht_offset': 0.0,
  'parts': 'chorus',
  'numpy': 'midi_encode/hooktheory/pianor

In [10]:
def format_meta(s):
    title = s['title'].title().replace('-', ' ')
    artist = s['artist'].title().replace('-', ' ')
    display = ' - '.join([title, artist])
    if s.get('section'): display += ' - ' + s['section'].title()
    sid = s['md5']
    
    json_data = {
        'title': title,
        'artist': artist,
        'bpm': s['ht_bpm'],
        'display': display,
        'genres': s['genres'],
        'sid': sid
    }
    source_file = file_path/data_dir/s['midi']
#     key_file = str(source_file).replace('cmajor', 'original')
    key_file = source_file
    to_file = encoded_path/f'{sid[::-1]}.mid'
    to_json = encoded_path/f'{sid[::-1]}.json'
    if not to_file.exists():
        to_file.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(str(key_file), str(to_file))
        with open(to_json, 'w') as jp:
            json.dump(json_data, jp)
    
    return json_data

In [11]:
def build_db(path):
    recordlist = df2records(path)
    htlist = [format_meta(s) for s in recordlist]
    json_path = file_path/'data/assets/v12/json/htlist.json'
    json_path.parent.mkdir(parents=True, exist_ok=True)
    with open(json_path, 'w') as fp:
        json.dump(htlist, fp, separators=(',', ':'))
    return htlist

In [12]:
htlist = build_db(path)

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (0,2,6,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
